In [1]:
# Importing necessary packages:
#import re
#import os
from glob import glob
#import math
import numpy as np
import pandas as pd
#import datetime as dt
#import itertools

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl
from matplotlib import rcParams, cycler
import matplotlib.lines as lines
from collections import OrderedDict
from matplotlib.legend import Legend
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from matplotlib.collections import PolyCollection # for chaing confidence interval color in statsmodels.graphics.tsaplots.plot_acf

import seaborn as sns

import statsmodels.tsa.stattools as tsas
import statsmodels.graphics.tsaplots as tsap

from polyphys.visualize import plotter
from polyphys.visualize import tuner
from polyphys.manage.parser import SumRule, TransFoci
from polyphys.manage import organizer
from polyphys.analyze import correlations

### Chain size vs $\phi_c$: 2021

In [ ]:
parent = '/Users/amirhsi_mini/analysis/'
properties_path = "all_in_one-properties-ens_avg-normalized.csv"
properties = pd.read_csv(parent+properties_path,header=0)

In [ ]:
# chain size plot
fontsize=20
properties_phi_c_trunc = properties[properties['phi_c_bulk_eff']<=0.4]
PipeLine.chainsize_plot(properties_phi_c_trunc, "phi_c_bulk", fontsize=fontsize)
PipeLine.chainsize_plot(properties_phi_c_trunc, "phi_c_bulk_eff", fontsize=fontsize)
PipeLine.chainsize_plot(properties_phi_c_trunc, "phi_c_bulk_normalized", fontsize=fontsize)
PipeLine.chainsize_plot(properties_phi_c_trunc, "phi_c_bulk_eff_normalized", fontsize=fontsize)

### The local distributions

In [ ]:
#direction_name = 'radial'
parent = '/Users/amirhsi_mini/analysis/'
simulation_type = 'all' # all or bug
attribute = 'distributions'
direction_name = 'radial'
direction = 'r'
dist_name = 'phi'
y_norm = '_norm'
x_norm_mon = ''#'_mon'
x_norm_crd = ''#'_crd'
distributions = pd.read_csv(parent+'all_in_one-'+simulation_type+'-'+attribute+'-'+direction_name+'-ens_avg.csv',index_col=0)
group_names = list(set(distributions.group_name))
sns.set_context('paper')
sns.set_style("ticks")
for group_name in group_names:
    group = distributions[(distributions.group_name==group_name )& (distributions.phi_c_bulk != 0)]
    fig, axes = plt.subplots(nrows=2,ncols=1,figsize=(16,12))
    sns.lineplot(x=direction+'_norm'+x_norm_mon, y=dist_name+"_mon_"+direction+y_norm, hue='phi_c_bulk', data=group,ax=axes[0],legend='full')
    sns.lineplot(x=direction+'_norm'+x_norm_crd, y=dist_name+"_crd_"+direction+y_norm, hue='phi_c_bulk', data=group,ax=axes[1],legend='full')
    fname = group_name+'-'+dist_name+'-'+direction+".pdf"
    plt.savefig(fname, dpi=200)
    plt.close()

### The sum rule

In [ ]:
parent = '/Users/amirhsi_mini/analysis/'
simulation_type = 'all' # all or bug
attribute = 'distributions'
direction_name = 'radial'
direction = 'r'
dist_name = 'phi'
y_norm = '_norm'
x_norm_mon = ''#'_mon'
x_norm_crd = ''#'_crd'
distributions = pd.read_csv(parent+'all_in_one-'+simulation_type+'-'+attribute+'-'+direction_name+'-ens_avg.csv',index_col=0)
group_names = list(set(distributions.group_name))
sns.set_context('paper')
sns.set_style("ticks")
for group_name in group_names:
    group = distributions[distributions.group_name==group_name]
    fig, axes = plt.subplots(nrows=2,ncols=1,figsize=(16,12))
    sns.lineplot(x=direction+'_norm'+x_norm_mon, y=dist_name+"_sumrule_"+direction, hue='phi_c_bulk', data=group,ax=axes[0],legend='full')
    sns.lineplot(x=direction+'_norm'+x_norm_crd, y=dist_name+"_sumrule_"+direction+'_norm', hue='phi_c_bulk', data=group,ax=axes[1],legend='full')
    fname = group_name+'-sum_rule-'+dist_name+'-'+direction+".pdf"
    plt.savefig(fname,dpi=200)
    plt.close()

In [ ]:
#cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["brown", "pink", "limegreen"])
flar_cmap = mpl.colors.ListedColormap(sns.cm._flare_lut)
flare_cmap_cut = truncate_colormap(flar_cmap, 0.5, 1.0, n=15)
mpl.cm.register_cmap("flare_cmap_cut", flare_cmap_cut)
binary_cmap = plt.get_cmap('binary')
binary_cmap_cut = truncate_colormap(binary_cmap, 0, 1, n=15)
mpl.cm.register_cmap("binary_cmap_cut", binary_cmap_cut)

In [ ]:
sns.color_palette("flare_cmap_cut", 15)

In [ ]:
sumrule_information = {
    'r' :
    {'direction_name':'radial',
     'xlabel':r'${2r}/{D}$',
     'legend_loc':'lower left',
     'legend_nloc':1},
    'z' : 
    {'direction_name':'longitudinal',
     'xlabel':r'${2z}/{z_{max}}$',
     'legend_loc':'lower left',
     'legend_nloc':1}
}
def plot_type_infomation(plot_type, direction):
    plot_type_dict = {
        'mon':{
            'yname':'mon',
            'ylabel':r'$\phi_m('+direction+')}{a_m}$'

        },
        'crd':{
            'yname':'crd',
            'ylabel':r'$\phi_c('+direction+')}{a_c}$'
        },
        'sumrule':{
            'yname':'sumrule',
            'ylabel':r'$\sum\frac{\phi_i('+direction+')}{a_i}$'
        }
    }
    return plot_type_dict[plot_type]

In [ ]:
direction = 'z'
plot_type = 'sumrule'
dist_name = 'phi'
x_norm_mon = ''#'_mon'
x_norm_crd = ''#'_crd'
plot_type_dict = plot_type_infomation(plot_type, direction)
#plot_type = '_sumrule_' #'_crd_', '_mon_', 'sumrule'
parent = '/Users/amirhsi_mini/analysis/'
simulation_type = 'all' # all or bug
attribute = 'distributions'
direction_name = 'longitudinal'
y_norm = '_norm'
distributions = pd.read_csv(parent+'all_in_one-'+simulation_type+'-'+attribute+'-'+direction_name+'-ens_avg.csv',index_col=0)
distributions = distributions.round({"phi_c_bulk":3})
group_names = list(set(distributions.group_name))
compression_rates = {}
for group_name in group_names:
    group_attributes = PipeLine.cellAttributes(group_name,'cylindrical',cell_type='group',warning=False)
    compression_rates[group_name]  = round(group_attributes.dcrowd/(group_attributes.dcyl-group_attributes.dcrowd),2)
compression_rates = dict(sorted(compression_rates.items(), key=lambda item: item[1]))
compression_rate_colors = sns.color_palette("binary_cmap_cut", len(group_names))#,as_cmap=True)

phi_c = list(set(distributions.phi_c_bulk))
phi_c = [0.1, 0.15, 0.2, 0.225, 0.25, 0.275, 0.3, 0.325, 0.35, 0.375, 0.4]
#phi_c_colors = sns.color_palette("PuRd", len(phi_c))#,as_cmap=True)
phi_c_colors = sns.color_palette("flare_cmap_cut", len(phi_c))#,as_cmap=True)
sns.set_context('paper')
sns.set_style("white")
if plot_type == 'sumrule':
    fig, axes = plt.subplots(nrows=4,ncols=3,figsize=(16,12),sharey=True)
else:
    fig, axes = plt.subplots(nrows=4,ncols=3,figsize=(16,12), sharey=True)#'row')
fontsize = 16
for idx, (group_name, ax) in enumerate(zip(compression_rates.keys(),axes.flat)):
    group_attributes =PipeLine.cellAttributes(group_name,'cylindrical',cell_type='group',warning=False)
    group = distributions[(distributions.group_name==group_name) & (distributions.phi_c_bulk!=0.0)]
    group_sumrule_rhd = distributions[(distributions.group_name==group_name) & (distributions.phi_c_bulk==0.0)]
    #ax.set_facecolor('gainsboro')
    ax.set_facecolor(compression_rate_colors[idx])
    #if  plot_type== 'sumrule':
    ax.plot(group_sumrule_rhd[direction+'_norm'+x_norm_crd], group_sumrule_rhd[dist_name+"_"+direction+'_uniform_sum_norm'],c='blue',ls='-')
    ax.grid(True,ls=':',lw=0.75,c='green')
    ax.tick_params(axis ='both',direction='inout',width=1,labelsize=fontsize-2,color='black')
    if plot_type != 'sumrule':
        if direction == 'r':
            #if idx > 5:
             #   PipeLine.yticks(ax,(0.0,2.5,0.25,0.05),code=True,fontsize=fontsize-2)
            #else : 
            if plot_type == 'crd':
                PipeLine.yticks(ax,(0.0,2.5,0.25,0.05),code=True,fontsize=fontsize-2)
            else :
                PipeLine.yticks(ax,(0.0,1.2,0.2,0.04),code=True,fontsize=fontsize-2)
            PipeLine.xticks(ax,(0.0,1,0.2,0.02),code=True,fontsize=fontsize-2)
        if direction == 'z':
            if plot_type == 'crd':
                pass
                #yticks(ax,(0.6,1.0,0.1,0.02),code=True,fontsize=fontsize-2)
            else:
                PipeLine.yticks(ax,(0.0,1.0,0.2,0.04),code=True,fontsize=fontsize-2)
            PipeLine.xticks(ax,(-1.0,1,0.2,0.04),code=True,fontsize=fontsize-2)
    ax.set_title(r"$({}):D={},a_c={},\kappa={}$".format(idx+1, group_attributes.dcyl,group_attributes.dcrowd,compression_rates[group_name]),fontsize=fontsize+2)
   
    ax = sns.lineplot(x=direction+'_norm'+x_norm_crd, y=dist_name+'_'+plot_type_dict['yname']+'_'+direction+'_norm', hue='phi_c_bulk', data=group,ax=ax,palette=phi_c_colors,legend=False)
    ax.set_ylabel(plot_type_dict['ylabel'],fontsize=fontsize+1)
    ax.set_xlabel(sumrule_information[direction]['xlabel'],fontsize=fontsize)
    
    
phi_c_patches = PipeLine.color_patcher(phi_c_colors)
phi_c_legend = Legend(axes[1,2],handles=phi_c_patches,labels= phi_c,title=r'$\phi_c^{(bulk)}$ (line color)',ncol=sumrule_information[direction]['legend_nloc'],
                      title_fontsize=fontsize-2,fontsize=fontsize-4,framealpha=None,frameon=True,loc=sumrule_information[direction]['legend_loc'],bbox_to_anchor=(1.05,-0.02),edgecolor='black')
axes[1,2].add_artist(phi_c_legend)
compression_rates_patches = PipeLine.color_patcher(compression_rate_colors)
compression_rate_legend = Legend(axes[3,2],handles=compression_rates_patches,labels= compression_rates.values(),title=r'$\kappa$ (background color)',ncol=sumrule_information[direction]['legend_nloc'], title_fontsize=fontsize-2,fontsize=fontsize-4,framealpha=None,frameon=True,loc=sumrule_information[direction]['legend_loc'],bbox_to_anchor=(1.05,-0.02),edgecolor='black')
axes[3,2].add_artist(compression_rate_legend)

if  plot_type== 'sumrule':
    sumrule_rhs_patches = mlines.Line2D([], [], color='blue', lw=2, ls='-', label=r'$\sum\frac{\phi_i}{a_i}=1$')
    axes[2,2].legend(handles=[sumrule_rhs_patches],fontsize=fontsize,frameon=True,bbox_to_anchor=(1.5,0.92),edgecolor='black')

fname = dist_name+'-'+plot_type+'-'+direction+".pdf"
fig.tight_layout()
plt.savefig(fname,dpi=200,bbox_inches='tight')
plt.close()

### End-to-end distribution

In [ ]:
#direction_name = 'radial'
parent = '/Users/amirhsi_mini/analysis/'
simulation_type = 'bug' # all or bug
attribute = 'rFloryHists'
direction = 'r'
y_norm = '_norm'
distributions = pd.read_csv(parent+'all_in_one-'+simulation_type+'-'+attribute+'-ens_avg.csv',index_col=0)
group_names = list(set(distributions.group_name))
sns.set_context('paper')
sns.set_style("ticks")

In [ ]:
for group_name in group_names:
    group = distributions[(distributions.group_name==group_name )& (distributions.phi_c_bulk != 0)]
    fig, axes = plt.subplots(nrows=2,ncols=1,figsize=(16,12))
    sns.lineplot(x=direction+'_norm'+x_norm_mon, y=dist_name+"_mon_"+direction+y_norm, hue='phi_c_bulk', data=group,ax=axes[0],legend='full')
    sns.lineplot(x=direction+'_norm'+x_norm_crd, y=dist_name+"_crd_"+direction+y_norm, hue='phi_c_bulk', data=group,ax=axes[1],legend='full')
    fname = group_name+'-'+dist_name+'-'+direction+".pdf"
    plt.savefig(fname,dpi=200)
    plt.close()